In [4]:
import sqlite3
import pandas as pd


def get_table_as_df(db, table):
    conn, df = (None, None)
    try:
        conn = sqlite3.connect(db)
        query = f"SELECT * FROM {table}"
        df = pd.read_sql_query(query, conn)
    except sqlite3.Error as error:
        print(f"Error while connecting to SQLite database {error}", error)
    except Exception as e:
        print(f"Error while reading data from {db}: {e}")
        return None
    finally:
        if conn:
            conn.close()
    return df



,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18
5,6,frontend engineer,20
6,7,iOS developer,27
7,8,android developer,27
8,9,machine learning engineer,35
9,10,ux/ui designer,15
